In [1]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AddChanneld,
    AsDiscrete,
    AsDiscreted,
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    EnsureType,
    Invertd,
    LabelFilterd,
    LoadImaged,
    RandFlipd,
    RandSpatialCropd,
    RandZoomd,
    Resized,
    ScaleIntensityRanged,
    SpatialCrop,
    SpatialCropd,
    ToTensord,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
from glob import glob

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import itk

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_Transforms import *


In [2]:
img1_dir = "../../Data/VFoldData/ColumnData/"

all_images = sorted(glob(os.path.join(img1_dir, '*Class[NS]*[0123456789].mha')))
all_labels = sorted(glob(os.path.join(img1_dir, '*Class[NS]*.roi-overlay.mha')))

print("# of Images =", len(all_images))
print("# of Labels =", len(all_labels))

num_classes = 2

num_columns = 16
num_slices = 48

num_workers_tr = 8
batch_size_tr = 32
num_workers_vl = 8
batch_size_vl = 4

model_filename_base = "BAMC_PTX_3DUNet-4Class.best_model.vfold64-Columns-ROI"

num_images = len(all_images)

num_folds = 10

ns_prefix = ['025ns','026ns','027ns','035ns','048ns','055ns','117ns',
             '135ns','193ns','210ns','215ns','218ns','219ns','221ns','247ns']
s_prefix = ['004s','019s','030s','034s','037s','043s','065s','081s',
            '206s','208s','211s','212s','224s','228s','236s','237s']

# of Images = 90
# of Labels = 90


In [3]:
fold_prefix_list = []
ns_count = 0
s_count = 0
for i in range(num_folds):
    if i%2 == 0:
        num_ns = 2
        num_s = 1
        if i > num_folds-3:
            num_s = 2
    else:
        num_ns = 1
        num_s = 2
    f = []
    for ns in range(num_ns):
        f.append([ns_prefix[ns_count+ns]])
    ns_count += num_ns
    for s in range(num_s):
        f.append([s_prefix[s_count+s]])
    s_count += num_s
    fold_prefix_list.append(f)
        
train_files = []
val_files = []
test_files = []
for i in range(num_folds):
    tr_folds = []
    for f in range(i,i+num_folds-2):
        tr_folds.append(fold_prefix_list[f%num_folds])
    tr_folds = list(np.concatenate(tr_folds).flat)
    va_folds = list(np.concatenate(fold_prefix_list[(i+num_folds-2) % num_folds]).flat)
    te_folds = list(np.concatenate(fold_prefix_list[(i+num_folds-1) % num_folds]).flat)
    train_files.append(
            [
                {"image": img, "label": seg}
                for img, seg in zip(
                    [im for im in all_images if any(pref in im for pref in tr_folds)],
                    [se for se in all_labels if any(pref in se for pref in tr_folds)])
            ]
        )
    val_files.append(
            [
                {"image": img, "label": seg}
                for img, seg in zip(
                    [im for im in all_images if any(pref in im for pref in va_folds)],
                    [se for se in all_labels if any(pref in se for pref in va_folds)])
            ]
        )
    test_files.append(
            [
                {"image": img, "label": seg}
                for img, seg in zip(
                    [im for im in all_images if any(pref in im for pref in te_folds)],
                    [se for se in all_labels if any(pref in se for pref in te_folds)])
            ]
        )
    print(len(train_files[i]),len(val_files[i]),len(test_files[i]))

70 14 6
74 6 10
72 10 8
73 8 9
73 9 8
74 8 8
75 8 7
76 7 7
70 7 13
63 13 14


In [4]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        ScaleIntensityRanged(keys=["image"],
            a_min=0, a_max=255,
            b_min=0.0, b_max=1.0),
        ARGUS_RandSpatialCropSlicesd(
            num_slices=num_slices,
            axis=2,
            keys=['image', 'label']),
        ARGUS_RandSpatialCropSlicesd(
            num_slices=num_columns,
            axis=0,
            require_labeled=True,
            keys=['image', 'label']),
        RandFlipd(prob=0.5, 
            spatial_axis=2,
            keys=['image', 'label']),
        RandFlipd(prob=0.5, 
            spatial_axis=0,
            keys=['image', 'label']),
        RandZoomd(prob=0.5, 
            min_zoom=1.0,
            max_zoom=1.2,
            keys=['image', 'label'],
            keep_size=True,
            mode=['trilinear', 'nearest']),
        ToTensord(keys=["image", "label"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        ScaleIntensityRanged(keys=["image"],
            a_min=0, a_max=255,
            b_min=0.0, b_max=1.0),
        ARGUS_RandSpatialCropSlicesd(
            num_slices=num_slices,
            axis=2,
            center_slice=30,
            keys=['image', 'label']),
        ARGUS_RandSpatialCropSlicesd(
            num_slices=num_columns,
            axis=0,
            center_slice=num_columns//2,
            keys=['image', 'label']),
        ToTensord(keys=["image", "label"]),
    ]
)

In [5]:
itkForceDynamicLoading = itk.imread(train_files[0][0]["image"])
print(itkForceDynamicLoading)
arrForceDynamicLoading = itk.GetArrayFromImage(itkForceDynamicLoading)

Image (0x56452c56cf50)
  RTTI typeinfo:   itk::Image<short, 3u>
  Reference Count: 1
  Modified Time: 442
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 246
  UpdateMTime: 441
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [93, 320, 61]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [93, 320, 61]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [93, 320, 61]
  Spacing: [1, 1, 1]
  Origin: [0, 0, 0]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
1 0 0
0 1 0
0 0 1

  PointToIndexMatrix: 
1 0 0
0 1 0
0 0 1

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x56452ce855f0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, short>
      Reference Count: 1
      Modified Time: 438
      Debug: Off
      Obj

In [6]:
train_ds = [CacheDataset(data=train_files[i], transform=train_transforms,cache_rate=1.0, num_workers=num_workers_tr)
            for i in range(num_folds)]
train_loader = [DataLoader(train_ds[i], batch_size=batch_size_tr, shuffle=True, num_workers=num_workers_tr) 
                for i in range(num_folds)]

val_ds = [CacheDataset(data=val_files[i], transform=val_transforms, cache_rate=1.0, num_workers=num_workers_vl)
          for i in range(num_folds)]
val_loader = [DataLoader(val_ds[i], batch_size=batch_size_vl, num_workers=num_workers_vl)
              for i in range(num_folds)]

Loading dataset: 100%|█████████████████████████| 13/13 [00:00<00:00, 110.66it/s]


In [7]:
device = torch.device("cuda:1")

In [8]:
def vfold_train(vfold_num, train_loader, val_loader):
    model = UNet(
        dimensions=3,
        in_channels=1,
        out_channels=num_classes,
        channels=(16, 32, 64),
        strides=(2, 2),
        num_res_units=2,
        norm=Norm.BATCH,
    ).to(device)
    loss_function = DiceLoss(to_onehot_y=True, softmax=True)
    optimizer = torch.optim.Adam(model.parameters(), 5e-5)
    dice_metric = DiceMetric(include_background=False, reduction="mean")


    max_epochs = 1000
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []
    post_pred = Compose([EnsureType(), AsDiscrete(argmax=True, to_onehot=True, num_classes=num_classes)])
    post_label = Compose([EnsureType(), AsDiscrete(to_onehot=True, num_classes=num_classes)])

    root_dir = "."

    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"{vfold_num}: epoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0
        for batch_data in train_loader:
            step += 1
            inputs, labels = (
                batch_data["image"].to(device),
                batch_data["label"].to(device),
            )
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            print(
                f"{step}/{len(train_ds) // train_loader.batch_size}, "
                f"train_loss: {loss.item():.4f}")
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"{vfold_num} epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                for val_data in val_loader:
                    val_inputs, val_labels = (
                        val_data["image"].to(device),
                        val_data["label"].to(device),
                    )
                    roi_size = (num_columns, 320, num_slices)
                    sw_batch_size = batch_size_vl
                    val_outputs = sliding_window_inference(
                        val_inputs, roi_size, sw_batch_size, model)
                    val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                    val_labels = [post_label(i) for i in decollate_batch(val_labels)]
                    # compute metric for current iteration
                    dice_metric(y_pred=val_outputs, y=val_labels)

                # aggregate the final mean dice result
                metric = dice_metric.aggregate().item()
                # reset the status for next validation round
                dice_metric.reset()

                metric_values.append(metric)
                if metric >= best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(
                        root_dir, model_filename_base+'_'+str(vfold_num)+'.pth'))
                    print("saved new best metric model")
                print(
                    f"current epoch: {epoch + 1} current mean dice: {metric:.8f}"
                    f"\nbest mean dice: {best_metric:.8f} "
                    f"at epoch: {best_metric_epoch}"
                )

    np.save(model_filename_base+"_loss_"+str(vfold_num)+".npy", epoch_loss_values)
    np.save(model_filename_base+"_val_dice_"+str(vfold_num)+".npy", metric_values)

In [9]:
for i in range(0,num_folds):
    vfold_train(i, train_loader[i], val_loader[i])


----------
0: epoch 1/1000
1/0, train_loss: 0.6636
2/0, train_loss: 0.6637
3/0, train_loss: 0.6513
0 epoch 1 average loss: 0.6595
----------
0: epoch 2/1000
1/0, train_loss: 0.6569
2/0, train_loss: 0.6534
3/0, train_loss: 0.6381
0 epoch 2 average loss: 0.6495
saved new best metric model
current epoch: 2 current mean dice: 0.26501149
best mean dice: 0.26501149 at epoch: 2
----------
0: epoch 3/1000
1/0, train_loss: 0.6501
2/0, train_loss: 0.6422
3/0, train_loss: 0.6588
0 epoch 3 average loss: 0.6504
----------
0: epoch 4/1000
1/0, train_loss: 0.6382
2/0, train_loss: 0.6435
3/0, train_loss: 0.6494
0 epoch 4 average loss: 0.6437
current epoch: 4 current mean dice: 0.26198176
best mean dice: 0.26501149 at epoch: 2
----------
0: epoch 5/1000
1/0, train_loss: 0.6405
2/0, train_loss: 0.6331
3/0, train_loss: 0.6258
0 epoch 5 average loss: 0.6331
----------
0: epoch 6/1000
1/0, train_loss: 0.6297
2/0, train_loss: 0.6351
3/0, train_loss: 0.6201
0 epoch 6 average loss: 0.6283
current epoch: 6 cur

1/0, train_loss: 0.5877
2/0, train_loss: 0.5972
3/0, train_loss: 0.5928
0 epoch 45 average loss: 0.5926
----------
0: epoch 46/1000
1/0, train_loss: 0.5879
2/0, train_loss: 0.5939
3/0, train_loss: 0.5603
0 epoch 46 average loss: 0.5807
current epoch: 46 current mean dice: 0.33904925
best mean dice: 0.34138811 at epoch: 44
----------
0: epoch 47/1000
1/0, train_loss: 0.5815
2/0, train_loss: 0.5989
3/0, train_loss: 0.5819
0 epoch 47 average loss: 0.5874
----------
0: epoch 48/1000
1/0, train_loss: 0.5818
2/0, train_loss: 0.5867
3/0, train_loss: 0.6700
0 epoch 48 average loss: 0.6128
saved new best metric model
current epoch: 48 current mean dice: 0.35545954
best mean dice: 0.35545954 at epoch: 48
----------
0: epoch 49/1000
1/0, train_loss: 0.5800
2/0, train_loss: 0.6108
3/0, train_loss: 0.5606
0 epoch 49 average loss: 0.5838
----------
0: epoch 50/1000
1/0, train_loss: 0.6093
2/0, train_loss: 0.5641
3/0, train_loss: 0.6052
0 epoch 50 average loss: 0.5929
current epoch: 50 current mean d

1/0, train_loss: 0.5673
2/0, train_loss: 0.5577
3/0, train_loss: 0.4742
0 epoch 90 average loss: 0.5331
saved new best metric model
current epoch: 90 current mean dice: 0.46315384
best mean dice: 0.46315384 at epoch: 90
----------
0: epoch 91/1000
1/0, train_loss: 0.5745
2/0, train_loss: 0.5368
3/0, train_loss: 0.4819
0 epoch 91 average loss: 0.5311
----------
0: epoch 92/1000
1/0, train_loss: 0.5230
2/0, train_loss: 0.5732
3/0, train_loss: 0.5245
0 epoch 92 average loss: 0.5402
saved new best metric model
current epoch: 92 current mean dice: 0.46922526
best mean dice: 0.46922526 at epoch: 92
----------
0: epoch 93/1000
1/0, train_loss: 0.5374
2/0, train_loss: 0.5579
3/0, train_loss: 0.5578
0 epoch 93 average loss: 0.5510
----------
0: epoch 94/1000
1/0, train_loss: 0.5326
2/0, train_loss: 0.5588
3/0, train_loss: 0.5561
0 epoch 94 average loss: 0.5492
saved new best metric model
current epoch: 94 current mean dice: 0.47783282
best mean dice: 0.47783282 at epoch: 94
----------
0: epoch 

0 epoch 134 average loss: 0.5200
saved new best metric model
current epoch: 134 current mean dice: 0.62356025
best mean dice: 0.62356025 at epoch: 134
----------
0: epoch 135/1000
1/0, train_loss: 0.5173
2/0, train_loss: 0.5057
3/0, train_loss: 0.4224
0 epoch 135 average loss: 0.4818
----------
0: epoch 136/1000
1/0, train_loss: 0.4793
2/0, train_loss: 0.5259
3/0, train_loss: 0.5346
0 epoch 136 average loss: 0.5133
current epoch: 136 current mean dice: 0.59109432
best mean dice: 0.62356025 at epoch: 134
----------
0: epoch 137/1000
1/0, train_loss: 0.5039
2/0, train_loss: 0.5028
3/0, train_loss: 0.5070
0 epoch 137 average loss: 0.5046
----------
0: epoch 138/1000
1/0, train_loss: 0.4837
2/0, train_loss: 0.5464
3/0, train_loss: 0.4016
0 epoch 138 average loss: 0.4772
current epoch: 138 current mean dice: 0.58793759
best mean dice: 0.62356025 at epoch: 134
----------
0: epoch 139/1000
1/0, train_loss: 0.4985
2/0, train_loss: 0.5163
3/0, train_loss: 0.4717
0 epoch 139 average loss: 0.4955

0 epoch 179 average loss: 0.4866
----------
0: epoch 180/1000
1/0, train_loss: 0.4999
2/0, train_loss: 0.4547
3/0, train_loss: 0.4409
0 epoch 180 average loss: 0.4652
current epoch: 180 current mean dice: 0.66365993
best mean dice: 0.69653034 at epoch: 164
----------
0: epoch 181/1000
1/0, train_loss: 0.5092
2/0, train_loss: 0.4529
3/0, train_loss: 0.3767
0 epoch 181 average loss: 0.4463
----------
0: epoch 182/1000
1/0, train_loss: 0.4561
2/0, train_loss: 0.4986
3/0, train_loss: 0.4170
0 epoch 182 average loss: 0.4573
current epoch: 182 current mean dice: 0.54064280
best mean dice: 0.69653034 at epoch: 164
----------
0: epoch 183/1000
1/0, train_loss: 0.4650
2/0, train_loss: 0.4659
3/0, train_loss: 0.5190
0 epoch 183 average loss: 0.4833
----------
0: epoch 184/1000
1/0, train_loss: 0.5019
2/0, train_loss: 0.4360
3/0, train_loss: 0.4685
0 epoch 184 average loss: 0.4688
current epoch: 184 current mean dice: 0.60351628
best mean dice: 0.69653034 at epoch: 164
----------
0: epoch 185/100

...

1/0, train_loss: 0.3525
2/0, train_loss: 0.3230
9 epoch 758 average loss: 0.3377
current epoch: 758 current mean dice: 0.73236841
best mean dice: 0.74640244 at epoch: 748
----------
9: epoch 759/1000
1/0, train_loss: 0.3434
2/0, train_loss: 0.3459
9 epoch 759 average loss: 0.3447
----------
9: epoch 760/1000
1/0, train_loss: 0.3677
2/0, train_loss: 0.3050
9 epoch 760 average loss: 0.3363
current epoch: 760 current mean dice: 0.74091524
best mean dice: 0.74640244 at epoch: 748
----------
9: epoch 761/1000
1/0, train_loss: 0.3736
2/0, train_loss: 0.3052
9 epoch 761 average loss: 0.3394
----------
9: epoch 762/1000
1/0, train_loss: 0.3423
2/0, train_loss: 0.3416
9 epoch 762 average loss: 0.3420
current epoch: 762 current mean dice: 0.74339539
best mean dice: 0.74640244 at epoch: 748
----------
9: epoch 763/1000
1/0, train_loss: 0.3326
2/0, train_loss: 0.3437
9 epoch 763 average loss: 0.3381
----------
9: epoch 764/1000
1/0, train_loss: 0.3555
2/0, train_loss: 0.3304
9 epoch 764 average lo

1/0, train_loss: 0.3771
2/0, train_loss: 0.2834
9 epoch 811 average loss: 0.3302
----------
9: epoch 812/1000
1/0, train_loss: 0.3403
2/0, train_loss: 0.3340
9 epoch 812 average loss: 0.3372
saved new best metric model
current epoch: 812 current mean dice: 0.75000823
best mean dice: 0.75000823 at epoch: 812
----------
9: epoch 813/1000
1/0, train_loss: 0.3501
2/0, train_loss: 0.3020
9 epoch 813 average loss: 0.3261
----------
9: epoch 814/1000
1/0, train_loss: 0.3338
2/0, train_loss: 0.3248
9 epoch 814 average loss: 0.3293
current epoch: 814 current mean dice: 0.74706423
best mean dice: 0.75000823 at epoch: 812
----------
9: epoch 815/1000
1/0, train_loss: 0.3127
2/0, train_loss: 0.3623
9 epoch 815 average loss: 0.3375
----------
9: epoch 816/1000
1/0, train_loss: 0.3523
2/0, train_loss: 0.2964
9 epoch 816 average loss: 0.3243
current epoch: 816 current mean dice: 0.74155271
best mean dice: 0.75000823 at epoch: 812
----------
9: epoch 817/1000
1/0, train_loss: 0.3431
2/0, train_loss: 0

1/0, train_loss: 0.3413
2/0, train_loss: 0.3166
9 epoch 864 average loss: 0.3289
current epoch: 864 current mean dice: 0.73426926
best mean dice: 0.75806177 at epoch: 824
----------
9: epoch 865/1000
1/0, train_loss: 0.3101
2/0, train_loss: 0.3376
9 epoch 865 average loss: 0.3239
----------
9: epoch 866/1000
1/0, train_loss: 0.3159
2/0, train_loss: 0.3335
9 epoch 866 average loss: 0.3247
current epoch: 866 current mean dice: 0.73218596
best mean dice: 0.75806177 at epoch: 824
----------
9: epoch 867/1000
1/0, train_loss: 0.3511
2/0, train_loss: 0.2903
9 epoch 867 average loss: 0.3207
----------
9: epoch 868/1000
1/0, train_loss: 0.3010
2/0, train_loss: 0.3600
9 epoch 868 average loss: 0.3305
current epoch: 868 current mean dice: 0.73246431
best mean dice: 0.75806177 at epoch: 824
----------
9: epoch 869/1000
1/0, train_loss: 0.3125
2/0, train_loss: 0.3386
9 epoch 869 average loss: 0.3255
----------
9: epoch 870/1000
1/0, train_loss: 0.3257
2/0, train_loss: 0.3256
9 epoch 870 average lo

1/0, train_loss: 0.3527
2/0, train_loss: 0.2842
9 epoch 917 average loss: 0.3184
----------
9: epoch 918/1000
1/0, train_loss: 0.3075
2/0, train_loss: 0.3393
9 epoch 918 average loss: 0.3234
current epoch: 918 current mean dice: 0.74314106
best mean dice: 0.75897998 at epoch: 900
----------
9: epoch 919/1000
1/0, train_loss: 0.3579
2/0, train_loss: 0.2933
9 epoch 919 average loss: 0.3256
----------
9: epoch 920/1000
1/0, train_loss: 0.3360
2/0, train_loss: 0.3006
9 epoch 920 average loss: 0.3183
current epoch: 920 current mean dice: 0.74993175
best mean dice: 0.75897998 at epoch: 900
----------
9: epoch 921/1000
1/0, train_loss: 0.3537
2/0, train_loss: 0.2958
9 epoch 921 average loss: 0.3248
----------
9: epoch 922/1000
1/0, train_loss: 0.2792
2/0, train_loss: 0.3607
9 epoch 922 average loss: 0.3200
current epoch: 922 current mean dice: 0.75408244
best mean dice: 0.75897998 at epoch: 900
----------
9: epoch 923/1000
1/0, train_loss: 0.3463
2/0, train_loss: 0.3026
9 epoch 923 average lo

2/0, train_loss: 0.3134
9 epoch 969 average loss: 0.3175
----------
9: epoch 970/1000
1/0, train_loss: 0.3564
2/0, train_loss: 0.2914
9 epoch 970 average loss: 0.3239
current epoch: 970 current mean dice: 0.75060135
best mean dice: 0.76812541 at epoch: 950
----------
9: epoch 971/1000
1/0, train_loss: 0.3063
2/0, train_loss: 0.3349
9 epoch 971 average loss: 0.3206
----------
9: epoch 972/1000
1/0, train_loss: 0.3552
2/0, train_loss: 0.2863
9 epoch 972 average loss: 0.3207
current epoch: 972 current mean dice: 0.75280714
best mean dice: 0.76812541 at epoch: 950
----------
9: epoch 973/1000
1/0, train_loss: 0.3316
2/0, train_loss: 0.3139
9 epoch 973 average loss: 0.3227
----------
9: epoch 974/1000
1/0, train_loss: 0.3359
2/0, train_loss: 0.2973
9 epoch 974 average loss: 0.3166
current epoch: 974 current mean dice: 0.74694401
best mean dice: 0.76812541 at epoch: 950
----------
9: epoch 975/1000
1/0, train_loss: 0.3234
2/0, train_loss: 0.3264
9 epoch 975 average loss: 0.3249
----------
9: